In [1]:
from pathlib import Path

from model_client import ModelClient, get_prompt
from pdf_extraction import extract_text_from_pdf

In [ ]:
pdf_path = Path("./data/pdfs")
client = ModelClient()

responses = []
for pdf in list(pdf_path.glob("*.pdf"))[:1]:
    print(pdf)
    paper_text = extract_text_from_pdf(pdf)
    print(paper_text)
    prompt = get_prompt(paper_text)
    response = client.ask_llama(prompt)
    responses.append(response)

data/pdfs/pdf (2).pdf
biosensors
Article
A Teenager Physical Fitness Evaluation Model Based on
1D-CNN with LSTM and Wearable Running PPG Recordings
JunqiGuo1,2,BoxinWan1,2,*,SiyuZheng3,AohuaSong1andWenshanHuang1
1 SchoolofArtificialIntelligence,BeijingNormalUniversity,Beijing100875,China;guojunqi@bnu.edu.cn(J.G.);
202121081008@mail.bnu.edu.cn(A.S.);202121081003@mail.bnu.edu.cn(W.H.)
2 EngineeringResearchCenterofIntelligentTechnologyandEducationalApplication,MinistryofEducation,
Beijing100875,China
3 SchoolofElectronicInformationandAutomation,TianjinUniversityofScienceandTechnology,
Tianjin300222,China;zhengsiyu0212@163.com
* Correspondence:202021210002@mail.bnu.edu.cn;Tel.:+86-130-3118-8188
Abstract:Peopleattachgreaterimportancetothephysicalhealthofteenagersbecauseadolescenceis
acriticalperiodforthehealthydevelopmentofthehumanbody. Withtheprogressofbiosensing
technologiesandartificialintelligence,itisfeasibletoapplywearabledevicestocontinuouslyrecord
teenagers’physiologicalsignalsandma

In [ ]:
print(responses[0])

Here are the results:


Paper Title: Elevated resting heart rate is an independent risk factor for cardiovascular disease in healthy men and women,
Decision: Exclude,
Reasons:
1. Wrong input data
2. Not human subjects
3. Not original research


Note: None of these reasons apply to this study, as it actually uses human subjects and has a significant sample size.
